# Práctica final de Normativa y Regulación

El desarrollo consiste en utilizar dos dataset o conjunto, para hacer uso de datos abiertos(Open Data) de datos del catalogo de datos [Open Data Canarias](http://www.opendatacanarias.es/) y [TENERIFEDATA](http://www.tenerifedata.es) en concreto el conjunto de datos [alojamientos turisticos de tenerife](http://www.opendatacanarias.es/datos/dataset/3f637e7d-d875-48c8-9337-6f16f767c655/resource/a6864f3e-acda-40fd-92a1-ae292e713399/download/alojamientos.csv) y el conjunto de datos [paradas de guaguas de tenerife](http://www.tenerifedata.es/dataset/transporte/resource/8e0664aa-e67d-47e0-adec-54b37fd21e38).
*****

**Características de estos conjuntos**
*****
* **Paradas de guaguas de Tenerife:**
    * **Resumen**: Recursos georeferenciados relacionados con la movilidad: aparcamientos, paradas de guaguas, ...
    * **Última actualización:** desconocido
    * **Fecha de creación:** 2 Años
    * **Formatos:** JSON,CSV
    * **Licencia:** [Creative Commons CCZero](http://opendefinition.org/licenses/cc-zero/)
    * **Autor:** desconocido
    * **Mantenedor:** desconocido
    * **Versión:** desconocido
    * ** Cobertura Geográfica:** Tenerife
    * **Idioma:** español
    *********
* **Alojamientos turísticos:**
    * **Resumen:** Contiene toda la información acerca de los alojamientos registrados por Turismo de Tenerife. La información que aporta este dataset es recogida desde el punto de vista de interés para el turista por lo que puede haber algún que no esté correcto. Los recursos se encuentran en los formatos: XML, JSON, CSV.<br>
    **NOTA:** Los ficheros en RDF se han realizado en colaboración con el grupo TARO de investigación de la Universidad de La Laguna.
    * **Última actualización:** Julio 3, 2015
    * ** Fecha de creación:** Junio 18,2014
    * **Formatos:** CSV, JSON, XML, RDF
    * **Licencia:** [Open Data Commons Attribution License](http://opendefinition.org/licenses/odc-by/)
    * **Autor:** SPET, Turismo de Tenerife S.A
    * **Mantenedor:** SPET, Turismo de Tenerife S.A
    * **Versión:** 1.1
    * ** Cobertura Geográfica:** Tenerife
    * **Idioma:** español  
    

## Aplicación
* ** Autor:** [Orlandy Ariel Sánchez A.](https://github.com/alu0100773408)
* ** Asignatura:** Normativa y Regulación.
* ** Lenguaje: ** Python
* **Licencia:** [Creative Commons Reconocimiento 4.0 Internacional](https://creativecommons.org/licenses/by/4.0/)

Para el desarrollo de la aplicación se hará uso de datos abiertos (Open Data) utilizando la herramienta [Jupyter Notebook](http://jupyter.org/).
Haciendo uso de los siguientes lenguajes y librerias.
* Para el texto se utilizará [Markdown](https://es.wikipedia.org/wiki/Markdown)
* Para el código [Python](https://es.wikipedia.org/wiki/Python)
* Se utilizarán las librerías:
    * [IPython](https://ipython.org/)
    * [gpxpy](https://pypi.python.org/pypi/gpxpy/0.7.0)
    * [Pandas](http://pandas.pydata.org/)
    * [Folium](https://folium.readthedocs.io/en/latest/)
    
**NOTA: TENER EN CUENTA DE QUE LAS LIBRERÍAS TIENEN QUE ESTAR INSTALADAS EN LA EN LA MÁQUINA PARA PODER EJECUTARLAS** 

### Definición de la aplicación
Se pretende leer de dos ficheros; uno contiene la ubicación de las paradas de guagua de todo Tenerife y el otro contiene la ubicación de los alojamientos turísticos de Tenerife. Luego de leer estos datos se procesarán y se calculará las paradas de guaguas que tienen más cerca un alojamiento turístico y se macará en el mapa.

### Paradas de guaguas de Tenerife

Para poder hacer uso de las librerías mencionadas antes se deben importar, esto se hace mediante la palabra reservada _**import**_

In [1]:
from IPython.display import HTML
import gpxpy.geo
import pandas as pd
import folium

El contenido de los ficheros, en este caso [**csv**](https://es.wikipedia.org/wiki/CSV), se guardará en la variable **datosParadas** y **datosAlojamientos** para leerlo se hace con la librería pandas. Si se necesita leer de un fichero que esté alojado en un servidor también se puede de la misma manera, solo que sustituyendo la ruta relativa del fichero por la url donde esté almacenado en el servidor.

**NOTA:** en comentarios está como se haría si se lee desde una url.

In [2]:
#Carga de datos paradas
#datosParadas = pd.read_csv('http://www.opendatacanarias.es/datos/dataset/3f637e7d-d875-48c8-9337-6f16f767c655/resource/a6864f3e-acda-40fd-92a1-ae292e713399/download/alojamientos.csv')
datosParadas = pd.read_csv('paradas.csv')

#Carga de datos alojamietnos.
#datosAlojamientos = pd.read_csv('http://www.tenerifedata.com/dataset/54c130a0-935a-4887-a963-35670f9091ec/resource/8e0664aa-e67d-47e0-adec-54b37fd21e38/download/paradas.csv')
datosAlojamientos = pd.read_csv('alojamientos.csv')

Para poder representar mapas utilizaremos la librería folium, representaremos 3 mapas:
1. Mapa en el que se guardarán la ubucación de los alojamientos con paradas cercanas, se guardará en la variable **mapaDistancias**.
2. Mapa en el que se guardarán la ubicación los alojamientos de Tenerife, se guardará en la variable **mapaAlojamientos**.
3. Mapa en el que se guardarán la ubicación de las paradas de guaguas de Tenerife, se guardará en la variable **mapaDistancias**.

La variable _location_, sitúa el mapa en la Isla de Santa Cruz de Tenerife

In [3]:
localizacion=[28.2915637, -16.6291304]
mapaParadas = folium.Map(localizacion, tiles='Stamen Terrain', zoom_start=9)
mapaAlojamientos = folium.Map(localizacion, tiles='Stamen Terrain', zoom_start=9)
mapaDistancias = folium.Map(localizacion, tiles='Stamen Terrain', zoom_start=9)

Necesitamos utilizar datos de tipo float, como no podemos asegurar que los datos del fichero csv están en correcto formato debemos comprobar que son float, para ello se utiliza la función isfloat, esta recibe una variable por parámetro. Si el valor de la variable es de tipo float entonces retorna _true_, sino devuelve _false_.

In [4]:
#Verificación que los datos sean de tipo float
def isfloat(value):
      try:
        float(value)
        return True
      except ValueError:
        return False

** Método para calcular la distancia entre dos puntos, haciendo uso de la librería gpxpy.geo que hace uso de la [Fórmula del haversine](https://es.wikipedia.org/wiki/F%C3%B3rmula_del_haversine). Este método devuelve la distancia en métros, y para no cargar el mapa se decidio para las paradas que esten a menos de 100 metros de los alojamientos.**

In [5]:
arrayCoordenadasAlojamientos = []
arrayCoordenadasParadas = []

def distancia(lat1, lng1,nom1, lat2, lng2,nom2):
    dist = gpxpy.geo.haversine_distance(lat1, lng1, lat2, lng2)
    if(dist < 100): # menor de 100 metros
        arrayCoordenadasParadas.append([lat1,lng1,nom1])
        arrayCoordenadasAlojamientos.append([lat2,lng2,nom2])

Para poder calcular las distancias, primero procesamos la información del fichero de alojamientos, quedándonos con lo que nos interesa, la ubicación (en coordenadas) y el nombre para posteriormente utilizarlo a la hora de representarlo en el mapa.

In [6]:
#extracción de los datos de las coordenadas
arrayAloj = []
for eq in datosAlojamientos.iterrows():
        coordenadas = eq[1]['ows_Georeferencia']
        if(not isfloat(coordenadas)):
            if(not coordenadas == ""):
                line = coordenadas.strip("(").strip(")").strip(" ")#Quitamos los "( )" y los espacios en blanco de las coordenadas
                line = line.replace(" ", "").split(",")#separamos las coordenadas mediante una ","
                nombre = eq[1]['ows_LinkTitle']
                line.append(nombre)#se le añade el nombre para tenerlo todo en un mismo array.
                arrayAloj.append(line)
arrayAlojamientos = []
for infoAloj in arrayAloj:
    x = float(infoAloj[0])
    y = float(infoAloj[1])
    arrayAlojamientos.append([x,y,infoAloj[2]])

Luego realizamos el procedimiento anterior pero ahora con la ubicación de las paradas de guagua.

In [7]:
arrayParadas = []
for eq in datosParadas.iterrows():
    if(isfloat(eq[1][1])):
        x= float(eq[1][1])
    if(isfloat(eq[1][0])):
        y=float(eq[1][0])
    arrayParadas.append([x,y,eq[1][3]])

Una vez tenemos los datos procesados podemos calcular la distancia entre las ubicaciones, es decir, mediante dos bucles for se recorrerán cada una de las paradas de guaguas y se verá cual/es alojamiento/os están cerca. Aquí se llama a la función distancia, definida un poco más arriba.

In [8]:
for i in range(len(arrayParadas)):
    for j in range(len(arrayAlojamientos)):
        distancia(arrayParadas[i][0],arrayParadas[i][1],arrayParadas[i][2],arrayAlojamientos[j][0],arrayAlojamientos[j][1],arrayAlojamientos[j][2])

Cuando ya tenemos las ubicaciones cercanas calculadas, pasamos a mostrarlas en el mapa. Aquí hacemos uso de la librería (importada previamente) folium. Se añaden marcas en el mapa y se guardan en el fichero _**HTML**_ llamado _**distancia.html**_ para poderlo mostrar más adelante.

In [9]:
icon_url_alojamiento = 'iconoAlojamiento.png'
icon_url_guagua = 'iconoGuagua.png'
icon_size=(40, 40)
for i in range(len(arrayCoordenadasParadas)):
    marker1 = folium.Marker(location=[arrayCoordenadasParadas[i][0],arrayCoordenadasParadas[i][1]],
                            popup = arrayCoordenadasParadas[i][2],
                            icon = folium.features.CustomIcon(icon_url_guagua,icon_size))
    marker2 = folium.Marker(location=[arrayCoordenadasAlojamientos[i][0],arrayCoordenadasAlojamientos[i][1]],
                            popup = arrayCoordenadasAlojamientos[i][2],
                            icon = folium.features.CustomIcon(icon_url_alojamiento,icon_size))
    mapaDistancias.add_children(marker1)
    mapaDistancias.add_children(marker2)
    
mapaDistancias.save('mapaDeDistancia.html')

Para poder ver el resultado de lo hecho anteriormente pasaremos a mostrar el mapa con los datos obtenidos.

**Leyenda: En los mapas nos encontraremos con dos tipos de iconos:**

<img src="./iconLeyenda/iconoAlojamiento.png"align="left"> Indica que en dicha ubicación se encuentra un alojamiento turistico.

<img src="./iconLeyenda/iconoGuagua.png"align="left"> Indica que en dicha ubicación se encuentra una parada de guagua

In [10]:
HTML('<iframe src=mapaDeDistancia.html width=100% height=350></iframe>')

## Referencia visual de donde están las paradas y los alojamientos.

Dado que son muchas paradas y muchos alojamientos, mostrarlos juntos puede ser confuso y costoso de procesar, se ha decidido mostrar por separado los distintos alojamientos y las distintas paradas en el mapa. De manera que se pueda apreciar mejor la diferencia entre los datos cuando no se tiene en cuenta la distancia y cuando se tiene en cuenta.

En el primer caso se lee las coordenadas de los alojamientos, extraídos anteriormente del fichero, se guardan en un fichero _**HTML**_ llamado **_alojamientos.html_** para mostrarlo posteriormente.

In [11]:
for x in range(len(arrayAlojamientos)):
    marca = folium.Marker(location=[arrayAlojamientos[x][0],arrayAlojamientos[x][1]],
                          popup = arrayAlojamientos[x][2],
                          icon = folium.features.CustomIcon(icon_url_alojamiento,icon_size))
    mapaAlojamientos.add_children(marca)

mapaAlojamientos.save('mapaDeAlojamientos.html')

In [12]:
HTML('<iframe src=mapaDeAlojamientos.html width=100% height=350></iframe>')

En este caso se lee las coordenadas de las paradas de guagua, extraídos anteriormente del fichero de paradas, se guardan en un fichero _**HTML**_ llamado **_paradas.html_** para mostrarlo posteriormente.

In [13]:
for j in range(len(arrayParadas)):
    marca = folium.Marker(location=[arrayParadas[j][0],arrayParadas[j][1]],
                          popup = arrayParadas[j][2],
                          icon = folium.features.CustomIcon(icon_url_guagua,icon_size))
    mapaParadas.add_children(marca)

mapaParadas.save('mapaDeParadas.html')

In [14]:
HTML('<iframe src=mapaDeParadas.html width=100% height=350></iframe>')